In [6]:
%cd ..
%load_ext autoreload
%autoreload 2

from pykitti.tracking import KittiTrackingLabels
from datasets.PyKitti2Dataset import PyKitti2
from hugeica import *
from Mine import *
from hugeica import Adam_Lie
from Crop import crop

import imageio
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import meshplot as mp

from sklearn.neighbors import NearestNeighbors
from AutoCalibration import *


def log_results(log):

    metrics = [] 
    i  = 1
    sortby, ascending = "loss", False
    
    for r, interval_t, interval_r in log:

        Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
        t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

        metrics.append([Q_cos, t_l2])

        print(interval_r, interval_t)

    print("loss", np.asarray(metrics).mean(0), np.asarray(metrics).std(0))

    metrics = [] # 0.50 / 5
    i  = 1
    sortby, ascending = "mi", True
    
    for r, interval_t, interval_r in log:

        Q_cos = r[i].sort_values(sortby, ascending=ascending)["Q_cos"].values[-1]
        t_l2 = r[i].sort_values(sortby, ascending=ascending)["t_l2"].values[-1]

        metrics.append([Q_cos, t_l2])

    print("mi", np.asarray(metrics).mean(0), np.asarray(metrics).std(0))

    metrics = [] 
    i  = 1
    sortby, ascending = "mi", True
    
    for r, interval_t, interval_r in log:

        Q_cos = r[i]["Q_cos"].values[-1]
        t_l2 = r[i]["t_l2"].values[-1]

        metrics.append([Q_cos, t_l2])

    print("Converged", np.asarray(metrics).mean(0), np.asarray(metrics).std(0))


basedir = "/home/ios/data3/kitti/tracking/training"
seq = "0000"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Load Data

In [7]:
kitti = PyKitti2(basedir, seq, with_labels="box")   

image = kitti[0][1]
box = kitti[0][4][0].astype(int)
label = kitti[0][5][0]
label_num = KittiTrackingLabels.classes.index(label)
image = np.asarray(image).copy()

def box_to_image(image, box, border=2):
    x1,y1, x2, y2 = box
    inside = image[y1:y1+y2, x1:x1+x2, :].copy()
    image[y1:y1+y2, x1:x1+x2, :] = (255, 255, 255)
    image[y1+border:y1+y2-border, x1+border:x1+x2-border, :] = inside[border:-border, border:-border, :]
    plt.rcParams["figure.figsize"] = (200, 100)
    plt.imshow(image)
    plt.show()

def px_in_box(query_px, box):
    x1,y1, x2, y2 = box

    a = np.logical_and(query_px[:, 0] >= x1, 
                        query_px[:, 0] <= (x1 + x2)) 
    b = np.logical_and(query_px[:, 1] >= y1, 
                          query_px[:, 1] <= (y1 + y2))
    return np.logical_and(a, b)    
    
#box_to_image(image, box)

files 154


/home/ios/anaconda3/envs/pytorch/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
datas = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10)]

kitti1 = datas[0]
initRT = kitti1.calib.T_cam2_velo.copy()

files 154
files 447
files 233
files 144
files 314


/home/ios/anaconda3/envs/pytorch/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


files 297
files 270
files 800
files 390
files 803


In [9]:
# Kitti dataset is recalibrated on different days, so we can just test on sequences that have the same calibration
datas_test1 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(10,14)]

files 294
files 373
files 78
files 340


In [10]:
datas_test2 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(14,18)]

files 106
files 376
files 209
files 145


In [11]:
datas_test3 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(18,20)]

files 339
files 1059


In [12]:
datas_test4 = [PyKitti2(basedir, f"{i:04d}", with_labels="none") for i in range(20,21)]

files 837


In [13]:
datas_test = [datas_test1, datas_test2, datas_test3, datas_test4]

In [14]:
calib0 = datas_test1[0].calib.T_cam2_velo
calib1 = datas_test2[0].calib.T_cam2_velo
calib2 = datas_test3[0].calib.T_cam2_velo
calib3 = datas_test4[0].calib.T_cam2_velo

calib = [calib0, calib1, calib2, calib3]

datas_test

[[PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0010)[294],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0011)[373],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0012)[78],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0013)[340]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0014)[106],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0015)[376],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0016)[209],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0017)[145]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0018)[314],
  PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0019)[1059]],
 [PyKitti2(base_path=/home/ios/data3/kitti/tracking/training, sequence=0020)[837]]]

# Testing accuracy

## Accuracy with grad 

In [10]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, Mg=3, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()
    
    
    for j in range(t):
        
        interval_r, interval_t = randomRT(0.3, 2)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.17721082  0.07156258 -0.1202072 ] r:  [-0.9326909   0.48453533  0.11656838] 

#   0: nll: 0.52/0.72, mi:  0.463/-0.076, acc: 77.5/63.5, eps: 0.23357, R=[-0.99, 0.43, 0.06], t=0.23, Q_cos=1.08
#  30: nll: 0.69/0.61, mi: -0.130/ 0.592, acc: 55.0/66.5, eps: 0.04069, R=[-0.07, 0.54, 0.43], t=0.19, Q_cos=0.69
#  60: nll: 0.52/0.58, mi:  1.451/ 0.914, acc: 75.0/70.2, eps: 0.06200, R=[-0.31, 0.49, 0.06], t=0.13, Q_cos=0.58
Saved checkpoint
#  90: nll: 0.88/0.62, mi: -0.565/ 0.823, acc: 47.5/67.0, eps: 0.00903, R=[-0.15, 0.48, 0.1], t=0.12, Q_cos=0.51
# 120: nll: 0.68/0.54, mi: -0.003/ 1.124, acc: 72.5/74.5, eps: 0.02642, R=[-0.21, 0.24, 0.01], t=0.10, Q_cos=0.32
Saved checkpoint
# 150: nll: 0.48/0.53, mi:  0.844/ 0.805, acc: 70.0/73.0, eps: 0.02823, R=[0.06, 0.28, 0.2], t=0.07, Q_cos=0.35
# 180: nll: 0.81/0.56, mi:  0.191/ 1.026, acc: 67.5/71.5, eps: 0.03998, R=[-0.11, 0.11, 0.12], t=0.03, Q_cos=0.20
# 210: nll: 0.37/0.60, mi:  2.495/ 0.787, acc: 80.0/7

In [11]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, Mg=3, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()
    
    
    for j in range(t):
        
        interval_r, interval_t = randomRT(0.6, 2)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-0.9326909   0.48453533  0.11656838] 

#   0: nll: 0.81/0.78, mi: -0.748/-0.198, acc: 55.0/61.3, eps: 0.45833, R=[-0.99, 0.43, 0.17], t=0.46, Q_cos=1.09
#  30: nll: 0.51/0.76, mi:  1.001/-0.021, acc: 77.5/64.0, eps: 0.02214, R=[-0.98, 0.69, -0.03], t=0.44, Q_cos=1.20
#  60: nll: 0.83/0.69, mi: -0.047/ 0.285, acc: 57.5/67.7, eps: 0.06988, R=[-0.85, 1.21, -0.19], t=0.36, Q_cos=1.50
#  90: nll: 0.73/0.67, mi: -0.040/ 0.560, acc: 57.5/68.2, eps: 0.01535, R=[-0.59, 1.15, -0.21], t=0.35, Q_cos=1.31
Saved checkpoint
# 120: nll: 0.55/0.67, mi:  0.708/ 0.329, acc: 70.0/67.0, eps: 0.05765, R=[-0.59, 0.93, -0.28], t=0.41, Q_cos=1.13
# 150: nll: 0.58/0.60, mi:  0.516/ 0.560, acc: 80.0/70.2, eps: 0.06507, R=[-0.24, 1.22, -0.63], t=0.34, Q_cos=1.39
# 180: nll: 0.56/0.54, mi:  0.834/ 1.006, acc: 62.5/73.5, eps: 0.04057, R=[0.2, 1.01, -0.35], t=0.30, Q_cos=1.09
Saved checkpoint
# 210: nll: 0.52/0.61, mi:  1.467/ 0.705, acc:

In [17]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, Mg=3, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()
    
    
    for j in range(t):
        
        interval_r, interval_t = randomRT(0.3, 5)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.17721082  0.07156258 -0.1202072 ] r:  [-2.33172725  1.21133833  0.29142094] 

#   0: nll: 0.66/0.81, mi:  0.308/-0.181, acc: 55.0/61.3, eps: 0.23494, R=[-2.27, 1.27, 0.35], t=0.23, Q_cos=2.63
#  30: nll: 0.81/0.74, mi: -0.362/-0.151, acc: 57.5/62.5, eps: 0.03065, R=[-2.28, 1.2, -0.04], t=0.19, Q_cos=2.58
#  60: nll: 0.88/0.74, mi: -0.581/ 0.053, acc: 47.5/60.7, eps: 0.01006, R=[-1.91, 1.14, -0.15], t=0.19, Q_cos=2.23
#  90: nll: 0.96/0.53, mi: -0.593/ 0.848, acc: 57.5/70.2, eps: 0.05275, R=[-1.14, 0.82, 0.1], t=0.24, Q_cos=1.41
Saved checkpoint
# 120: nll: 0.67/0.62, mi:  0.443/ 0.225, acc: 62.5/69.5, eps: 0.05217, R=[-0.68, 0.55, -0.04], t=0.30, Q_cos=0.87
# 150: nll: 0.75/0.61, mi:  0.182/ 0.762, acc: 65.0/67.2, eps: 0.09769, R=[-0.77, 0.89, 0.2], t=0.20, Q_cos=1.19
# 180: nll: 0.66/0.56, mi:  0.252/ 0.764, acc: 60.0/72.3, eps: 0.00696, R=[-0.6, 0.55, 0.23], t=0.21, Q_cos=0.85
# 210: nll: 0.49/0.59, mi:  0.940/ 0.788, acc: 72.5/69.0, eps: 0.002

In [19]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, Mg=3, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()
    
    
    for j in range(t):
        
        interval_r, interval_t = randomRT(0.6, 5)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-2.33172725  1.21133833  0.29142094] 

#   0: nll: 0.97/0.79, mi: -0.514/-0.337, acc: 52.5/56.2, eps: 0.44688, R=[-2.39, 1.27, 0.35], t=0.44, Q_cos=2.73
#  30: nll: 0.69/0.78, mi:  0.170/-0.214, acc: 65.0/57.5, eps: 0.05633, R=[-2.46, 0.94, -0.11], t=0.50, Q_cos=2.64
#  60: nll: 0.93/0.80, mi: -0.564/-0.295, acc: 67.5/52.0, eps: 0.00977, R=[-2.43, 1.06, -0.28], t=0.49, Q_cos=2.66
#  90: nll: 0.80/0.83, mi: -0.183/-0.707, acc: 55.0/55.5, eps: 0.06439, R=[-2.38, 0.9, -0.15], t=0.55, Q_cos=2.54
# 120: nll: 0.97/0.70, mi: -0.733/ 0.102, acc: 52.5/57.5, eps: 0.00410, R=[-2.28, 1.0, -0.7], t=0.56, Q_cos=2.59
# 150: nll: 0.72/0.73, mi: -0.025/-0.004, acc: 62.5/63.2, eps: 0.05786, R=[-1.85, 1.48, -0.65], t=0.50, Q_cos=2.45
# 180: nll: 0.75/0.64, mi: -0.134/ 0.259, acc: 57.5/68.5, eps: 0.04515, R=[-1.41, 1.11, -0.3], t=0.55, Q_cos=1.82
Saved checkpoint
# 210: nll: 0.62/0.67, mi:  0.319/ 0.332, acc: 70.0/68.0, eps: 0.

In [10]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, pc_batch=5000, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    for j in range(t):
        
        interval_r, interval_t = randomRT(0.3, 7)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.17721082  0.07156258 -0.1202072 ] r:  [-3.26441815  1.69587366  0.40798932] 

#   0: nll: 0.79/0.85, mi:  0.070/-0.588, acc: 57.5/48.0, eps: 0.24972, R=[-3.32, 1.64, 0.46], t=0.23, Q_cos=3.74
#  30: nll: 1.06/0.85, mi: -1.048/-0.353, acc: 42.5/54.0, eps: 0.04361, R=[-3.67, 1.28, 0.6], t=0.28, Q_cos=3.94
#  60: nll: 0.83/0.79, mi: -0.299/-0.158, acc: 62.5/55.3, eps: 0.07110, R=[-3.54, 1.78, 0.75], t=0.20, Q_cos=4.04
#  90: nll: 0.78/0.79, mi: -0.197/-0.244, acc: 50.0/57.0, eps: 0.00538, R=[-3.21, 1.67, 0.93], t=0.20, Q_cos=3.75
# 120: nll: 0.96/0.85, mi: -0.945/-0.434, acc: 50.0/57.7, eps: 0.01776, R=[-2.97, 1.39, 0.88], t=0.22, Q_cos=3.41
# 150: nll: 0.71/0.84, mi: -0.448/-0.268, acc: 65.0/60.0, eps: 0.00230, R=[-2.72, 1.33, 0.75], t=0.22, Q_cos=3.13
# 180: nll: 0.88/0.82, mi: -0.196/-0.096, acc: 62.5/63.0, eps: 0.01848, R=[-2.24, 1.46, 1.21], t=0.20, Q_cos=2.95
# 210: nll: 0.64/0.73, mi:  0.309/-0.274, acc: 62.5/62.8, eps: 0.00010, R=[-1.77, 1.4

In [11]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, pc_batch=5000, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    for j in range(t):
        
        interval_r, interval_t = randomRT(0.6, 7)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-3.26441815  1.69587366  0.40798932] 

#   0: nll: 0.73/0.80, mi: -0.102/-0.168, acc: 55.0/53.5, eps: 0.46209, R=[-3.32, 1.75, 0.46], t=0.45, Q_cos=3.79
#  30: nll: 1.36/0.91, mi: -1.818/-0.515, acc: 42.5/58.0, eps: 0.06041, R=[-3.08, 2.63, 1.03], t=0.39, Q_cos=4.19
#  60: nll: 0.72/0.89, mi:  0.195/-0.524, acc: 57.5/57.7, eps: 0.00994, R=[-2.83, 2.58, 1.09], t=0.38, Q_cos=4.00
#  90: nll: 0.93/0.91, mi: -0.924/-0.614, acc: 62.5/56.5, eps: 0.00337, R=[-2.45, 2.23, 0.93], t=0.38, Q_cos=3.45
# 120: nll: 0.72/0.93, mi: -0.028/-0.466, acc: 62.5/58.5, eps: 0.03228, R=[-2.08, 1.97, 1.02], t=0.41, Q_cos=3.06
# 150: nll: 0.75/0.76, mi:  0.156/-0.100, acc: 57.5/62.0, eps: 0.04606, R=[-2.04, 2.18, 0.59], t=0.37, Q_cos=3.05
# 180: nll: 0.65/0.89, mi:  0.202/-0.438, acc: 57.5/56.2, eps: 0.09749, R=[-2.32, 1.97, 0.67], t=0.47, Q_cos=3.12
# 210: nll: 0.73/0.92, mi:  0.199/-0.729, acc: 65.0/58.0, eps: 0.01123, R=[-1.98, 2.

In [10]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, pc_batch=5000, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    for j in range(t):
        
        interval_r, interval_t = randomRT(0.3, 8)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.17721082  0.07156258 -0.1202072 ] r:  [-3.7307636   1.93814132  0.46627351] 

#   0: nll: 0.74/0.80, mi:  0.143/-0.195, acc: 67.5/57.0, eps: 0.24797, R=[-3.67, 1.88, 0.41], t=0.23, Q_cos=4.15
#  30: nll: 0.62/0.73, mi:  0.444/ 0.058, acc: 60.0/61.7, eps: 0.11132, R=[-2.6, 1.3, 0.19], t=0.35, Q_cos=2.92
#  60: nll: 0.70/0.71, mi:  0.438/ 0.131, acc: 62.5/63.7, eps: 0.02510, R=[-1.97, 1.16, 0.66], t=0.38, Q_cos=2.38
Saved checkpoint
#  90: nll: 0.85/0.57, mi: -0.193/ 0.929, acc: 52.5/72.2, eps: 0.03728, R=[-1.31, 1.19, 0.92], t=0.42, Q_cos=2.00
Saved checkpoint
# 120: nll: 0.60/0.62, mi:  0.459/ 0.492, acc: 65.0/67.3, eps: 0.00332, R=[-1.58, 1.05, 0.63], t=0.42, Q_cos=2.00
# 150: nll: 0.71/0.56, mi:  0.218/ 0.565, acc: 70.0/72.5, eps: 0.03344, R=[-1.52, 0.91, 0.31], t=0.45, Q_cos=1.80
# 180: nll: 0.81/0.57, mi: -0.338/ 0.741, acc: 57.5/73.2, eps: 0.00241, R=[-1.48, 1.1, 0.29], t=0.45, Q_cos=1.86
# 210: nll: 0.66/0.61, mi:  0.364/ 0.717, acc: 60.0/6

In [12]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, pc_batch=5000, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    for j in range(t):
        
        interval_r, interval_t = randomRT(0.6, 8)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.35442164  0.14312516 -0.24041439] r:  [-3.7307636   1.93814132  0.46627351] 

#   0: nll: 1.21/0.91, mi: -1.559/-0.482, acc: 45.0/52.2, eps: 0.45807, R=[-3.67, 1.99, 0.41], t=0.45, Q_cos=4.21
#  30: nll: 1.28/0.78, mi: -1.773/-0.159, acc: 45.0/59.2, eps: 0.10872, R=[-3.43, 2.67, -0.04], t=0.33, Q_cos=4.34
#  60: nll: 0.86/0.82, mi: -0.555/-0.327, acc: 50.0/52.3, eps: 0.02708, R=[-3.12, 2.98, -0.29], t=0.30, Q_cos=4.32
#  90: nll: 1.16/0.82, mi: -1.199/-0.316, acc: 40.0/57.3, eps: 0.03393, R=[-3.24, 3.28, -0.47], t=0.26, Q_cos=4.62
# 120: nll: 0.77/0.89, mi:  0.072/-0.568, acc: 55.0/46.0, eps: 0.00521, R=[-3.48, 3.31, -0.35], t=0.27, Q_cos=4.81
# 150: nll: 0.97/0.75, mi: -0.490/-0.097, acc: 50.0/62.5, eps: 0.02906, R=[-3.38, 4.07, -0.2], t=0.23, Q_cos=5.29
# 180: nll: 0.79/0.82, mi: -0.241/-0.283, acc: 52.5/52.5, eps: 0.02691, R=[-3.43, 4.11, -0.48], t=0.20, Q_cos=5.37
# 210: nll: 0.76/0.76, mi: -0.138/-0.143, acc: 62.5/61.3, eps: 0.02012, R=[-3.1

In [13]:
log = []
np.random.seed(2)

t = 2
for i in [3,2,1,0]: 
        
    print("Using test set #", i)    
    datas_test_ = datas_test[i]
    kitti1 = datas_test_[0]
    
    
    ac = AutoCalibration(kitti1.calib.T_cam2_velo, kitti1.calib.P_rect_20, 
                         max_z=30, pc_batch=5000, M=20, mi_batches=1,
                         patch_size=96, hidden_size=32)
    
    
    ac.to("cuda")
    ac.load_state_dict(torch.load("checkpoints/mi_model_p96_small.pth.tar"))
    ac.MINE.load_state_dict(torch.load("checkpoints/mi_model_p96_small_2000/MINE_3999.pth.tar"))
    ac.reset_calibration()

    for j in range(t):
        
        interval_r, interval_t = randomRT(0.3, 9)
        print("\n", "Distortion", "t: ", interval_t, "r: ",  interval_r, "\n")
        ac.distort_calibration(interval_t, interval_r)
    
            
        for iters, lr, grad, sample in [(3000, 1e-3, False, False), (1000, 1e-4, True, False), (1000, 1e-5, True, False)]:

            ac.update_se_and_mine(lr_mi=lr, lr_R=lr, lr_T=10*lr)
            ac.best_mi = 0.
            r = ac.fit(datas_test_, iters=iters, log_interval=30, viz="none", eps=1e-8,  
                       log_avg_window=1,
                       n_test_batches=10, 
                       valid_ratio=0.1,
                       grad_mine=grad,
                       grad_SE=True, 
                       grad_R=True,
                       grad_T=True,
                       weight_pool_root="checkpoints/mi_model_p96_small_" + str(2000) + "mod", 
                       sample_weight_pool=sample)

        log.append((r, interval_t, interval_r))
    
log_results(log)

Using test set # 3

 Distortion t:  [-0.17721082  0.07156258 -0.1202072 ] r:  [-4.19710905  2.18040899  0.5245577 ] 

#   0: nll: 0.69/0.91, mi:  0.709/-0.537, acc: 75.0/51.7, eps: 0.24989, R=[-4.14, 2.24, 0.58], t=0.22, Q_cos=4.75
#  30: nll: 0.75/0.89, mi: -0.246/-0.425, acc: 55.0/56.2, eps: 0.04680, R=[-3.57, 2.09, 1.34], t=0.17, Q_cos=4.37
#  60: nll: 0.93/0.90, mi: -0.492/-0.575, acc: 55.0/56.7, eps: 0.01982, R=[-3.25, 2.13, 1.63], t=0.20, Q_cos=4.23
#  90: nll: 0.66/0.83, mi:  0.530/-0.254, acc: 67.5/65.0, eps: 0.15703, R=[-2.89, 1.83, 1.61], t=0.37, Q_cos=3.80
# 120: nll: 0.72/0.75, mi: -0.017/-0.053, acc: 52.5/65.2, eps: 0.07460, R=[-2.47, 1.64, 1.57], t=0.45, Q_cos=3.37
# 150: nll: 0.95/0.62, mi: -0.934/ 0.600, acc: 60.0/71.0, eps: 0.05113, R=[-2.15, 1.69, 2.2], t=0.39, Q_cos=3.52
# 180: nll: 0.51/0.66, mi:  0.798/ 0.350, acc: 72.5/69.3, eps: 0.03450, R=[-1.88, 1.36, 2.72], t=0.43, Q_cos=3.59
# 210: nll: 0.62/0.73, mi:  0.815/ 0.168, acc: 72.5/62.0, eps: 0.04325, R=[-1.91, 1.5